<a href="https://colab.research.google.com/github/B-Ridge99/Quantitative-Screener/blob/main/Copy_of_Swing_Trade_Market_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Required Packages
!pip install yfinance ta yahoo_fin pandas numpy

# Import Libraries
import yfinance as yf # For Price History, Technical Indicators (EMA, SMA, RSI, OBV, VWAP, Volume)
import pandas as pd # For Data Manipulation
import numpy as np # For Numerical Calculation
from ta import trend, momentum, volatility, volume # Technical Analysis Library for Indicators (RSI, OBV, MACD, etc.)
from yahoo_fin import stock_info as si # For Fundamental Indicators (PEG, EPS YoY, Revenue Growth, D/E, etc.)
import datetime

# Import Data (In This Case: S&P500)
sp500_tickers = si.tickers_sp500()

# Define Helper Functions

  # RSI Tiers
def classify_rsi(rsi):
    if rsi < 30:
        return "Oversold (<30)"
    elif 30 <= rsi < 50:
        return "Neutral (30-50)"
    elif 50 <= rsi < 70:
        return "Bullish Momentum (50-70)"
    else:
        return "Overbought (>70)"

  # Volume Profile
def get_volume_profile(df, bins=20):
    df = df.copy()
    df['Price Bin'] = pd.cut(df['Close'], bins)
    vp = df.groupby('Price Bin')['Volume'].sum().sort_values(ascending=False)
    return vp.index[0]  # Approx. Point of Control (POC)

# Scan Stocks
results = []

for ticker in sp500_tickers[:50]:  # limit for now; expand after testing
    try:
        stock = yf.download(ticker, period="6mo", interval="1d", progress=False)
        if stock.empty or len(stock) < 50:
            continue

        # Fundamentals
        stats = si.get_stats_valuation(ticker)
        summary = si.get_quote_table(ticker)

        pe_ratio = summary.get("PE Ratio (TTM)", np.nan)
        eps = summary.get("EPS (TTM)", np.nan)
        peg = float(stats.iloc[4, 1]) if not stats.empty else np.nan
        market_cap = summary.get("Market Cap", "")
        debt_equity = si.get_balance_sheet(ticker).loc["totalLiab"].iloc[0] / si.get_balance_sheet(ticker).loc["totalStockholderEquity"].iloc[0]

        # Revenue growth approx (not exact QoQ for now)
        income = si.get_income_statement(ticker)
        revenue_growth = income.loc["totalRevenue"].pct_change(periods=1).iloc[0]  # latest QoQ

        # Skip if fundamental filters not met
        if not (0.8 <= peg <= 1.5 and eps > 0 and revenue_growth > 0 and debt_equity < 1):
            continue

        # Calculate Indicators
        stock['EMA20'] = trend.ema_indicator(stock['Close'], window=20)
        stock['EMA50'] = trend.ema_indicator(stock['Close'], window=50)
        stock['SMA200'] = trend.sma_indicator(stock['Close'], window=200)
        stock['RSI'] = momentum.rsi(stock['Close'], window=14)
        stock['RSI Tier'] = stock['RSI'].apply(classify_rsi)
        stock['MACD'] = trend.macd_diff(stock['Close'])
        stock['ATR'] = volatility.average_true_range(stock['High'], stock['Low'], stock['Close'])
        stock['OBV'] = volume.on_balance_volume(stock['Close'], stock['Volume'])

        latest = stock.iloc[-1]

        vwap = (stock['Volume'] * (stock['High'] + stock['Low'] + stock['Close']) / 3).sum() / stock['Volume'].sum()
        volume_poc = get_volume_profile(stock)

        # Technical filter
        if latest['Close'] > latest['EMA20'] > latest['EMA50'] > latest['SMA200'] and \
           latest['MACD'] > 0 and \
           latest['Close'] > vwap:
            results.append({
                'Ticker': ticker,
                'Close': latest['Close'],
                'RSI': round(latest['RSI'], 2),
                'RSI Tier': classify_rsi(latest['RSI']),
                'MACD': round(latest['MACD'], 3),
                'ATR': round(latest['ATR'], 2),
                'OBV': latest['OBV'],
                'VWAP': round(vwap, 2),
                'Volume POC': volume_poc,
                'PEG': peg,
                'EPS': eps,
                'Revenue QoQ': round(revenue_growth, 2),
                'Debt/Equity': round(debt_equity, 2),
                'Passes Scan': 'Yes'
            })
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Export to CSV
df_results = pd.DataFrame(results)
df_results.to_csv("swing_scanner_results.csv", index=False)
df_results.head()



